In [5]:
import pandas as pd
import json
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import tifffile
import imagecodecs
from ipywidgets import interact
from pathlib import Path
from torchvision import models, transforms

# 1. 데이터셋 리스트 생성(label값 필터링(.json))

In [6]:
# 원본 경로
label_file_path = "./Dataset/01.데이터/1.Training/라벨링데이터_240326_add/콘크리트_콘크리트균열_라벨링_01/"
image_file_path = r"D:\Crack Detection\112.건물 균열 탐지드론 개발을 위한 이미지\01.데이터\1.Training\원천데이터\콘크리트_콘크리트균열_원천_03"
image_file_path = image_file_path.replace('\\','/')
image_file_path

'D:/Crack Detection/112.건물 균열 탐지드론 개발을 위한 이미지/01.데이터/1.Training/원천데이터/콘크리트_콘크리트균열_원천_03'

In [7]:
# 라벨 파일리스트 생성 -> lael_files
label_files = [fn for fn in os.listdir(label_file_path) if fn.endswith('json')]
print(len(label_files))
print(label_files[0])

144194
101_0000149e-4c13-49dd-a77b-c24b3d34a314.json


In [8]:
# 이미지 파일리스트 생성 -> image_files
image_files = [fn for fn in os.listdir(image_file_path) if fn.endswith('tiff')]
len(image_files)

FileNotFoundError: [WinError 3] 지정된 경로를 찾을 수 없습니다: 'D:/Crack Detection/112.건물 균열 탐지드론 개발을 위한 이미지/01.데이터/1.Training/원천데이터/콘크리트_콘크리트균열_원천_03'

In [ ]:
# image_files_파일명
image_files_name = [i.split('.')[0] for i in image_files]
image_files_name
# 이미지01_데이터셋에 맞는 label_files 리스트 생성
new_label_files = []
for i in label_files:
    if i.split('.')[0] in image_files_name:
        new_label_files.append(i)
# 리스트 정렬
new_label_files = sorted(new_label_files)
image_files = sorted(image_files)
len(new_label_files)

In [ ]:
# 함수1_make_file_list
def make_file_list(path, file):
    file_list = [os.path.join(path, i).replace('\\','/') for i in  file]
    return file_list

In [ ]:
# 데이터셋 파일경로 생성
label_file_list = make_file_list(label_file_path, new_label_files)
image_file_list = make_file_list(image_file_path, image_files)
print(len(label_file_list))
print(len(image_file_list))

In [ ]:
label_file_list[0]
image_file_list[0]

# 2. 데이터 체크

In [ ]:
# 인덱스별로 이미지 확인 함수
def show_sample(label_file_list, image_file_list, index):
    image = tifffile.imread(image_file_list[index])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # bbox좌표
    with open(label_file_list[index], 'r', encoding='utf-8') as f:
        data = json.load(f)
        bbox = [i['bbox'] for i in data['annotations']]
        bbox = [[int(i) for i in li] for li in bbox]
    print('image_name : ' + label_file_list[index].split('/')[-1].split('.')[0])
    print(bbox)
    for li in bbox:
        x,y,w,h = li
        cv2.rectangle(image, (x, y), (x+w, y+h), (255,0,0), 7)
    plt.figure(figsize=(6,6))
    plt.imshow(image)
    plt.show()

In [ ]:
# 인터랙티브 함수 정의
@interact(index=(0, len(label_file_list)-1))
def interact_image(index):
    show_sample(label_file_list, image_file_list, index)

# 3. image 데이터셋 생성(tiff -> jpg, 640)

In [ ]:
# # 이미지 변환 폴더생성
# new_folder_path = Path('./crack_dataset_0504/03_resized_images')
# new_folder_path.mkdir(parents=True, exist_ok=True)
# new_folder_path
# # 이미지 변환
# for idx, img in enumerate(image_file_list):
#     image = tifffile.imread(img)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     h,w,_ = image.shape
#     new_h = 640
#     new_w = int(w * (new_h / h))
#     resize_image = cv2.resize(image, (new_w,new_h), interpolation=cv2.INTER_CUBIC)
#     img_name = img.split('/')[-1].split('.')[0]
#     cv2.imwrite(os.path.join(new_folder_path,f'{img_name}.jpg'), resize_image)

# 4. label 데이터셋 생성(.txt / 클래스 번호 / bbox좌표_정규화)

In [ ]:
# # 라벨 폴더생성
# new_folder_path = Path('./crack_dataset_0504/03_resized_labels')
# new_folder_path.mkdir(parents=True, exist_ok=True)
# # 폴더 경로 설정
# txt_folder_path = new_folder_path
# # 라벨 .txt 파일 생성
# for file in label_file_list:
#     with open(file, 'r', encoding='utf-8') as f:
#         data = json.load(f)
#         bbox = [i['bbox'] for i in data['annotations']]
#         bbox = [[int(i) for i in li] for li in bbox]
        
#         # 이미지 크기 구하기
#         img_w = data['images'][0]['width']
#         img_h = data['images'][0]['height']
        
#         # 세로 너비를 기준으로 640 픽셀로 조정하면서 종횡비를 유지합니다.
#         scale_factor = 640 / max(img_w, img_h)
#         new_img_w = int(img_w * scale_factor)
#         new_img_h = int(img_h * scale_factor)
        
#         file_name = file.split('/')[-1].split('.')[0] + '.txt'
#         file_path = os.path.join(new_folder_path, file_name)

#         with open(file_path, 'w') as f:
#             for bbox_li in bbox:
#                 x, y, w, h = bbox_li
                
#                 # bbox 좌표를 새로운 크기에 맞게 조정합니다.
#                 x = (x + (w / 2)) * scale_factor / new_img_w
#                 y = (y + (h / 2)) * scale_factor / new_img_h
#                 w = w * scale_factor / new_img_w
#                 h = h * scale_factor / new_img_h
                
#                 bbox_list = [x, y, w, h]
#                 bbox_list = [float(i) for i in bbox_list]
#                 bbox_list = [0] + bbox_list
                
#                 result = " ".join(map(str, bbox_list))
#                 f.write(" ".join(map(str, bbox_list)) + "\n")

In [ ]:
# # 라벨 폴더생성
# new_folder_path = Path('./crack_dataset_0428/03_resized_labels')
# new_folder_path.mkdir(parents=True, exist_ok=True)
# # 폴더 경로 설정
# txt_folder_path = new_folder_path
# # 라벨 .txt파일 생성
# for file in label_file_list:
#     with open(file, 'r', encoding='utf-8') as f:
#             data = json.load(f)
#             bbox = [i['bbox'] for i in data['annotations']]
#             bbox = [[int(i) for i in li] for li in bbox]
#             # 이미지 크기 구하기
#             img_w = data['images'][0]['width']
#             img_h = data['images'][0]['height']
#             # # 이미지 변환 변수
#             # w_change = 640 / img_w
#             # h_change = 640 / img_h
#     file_name = file.split('/')[-1].split('.')[0] + '.txt'
#     file_path = os.path.join(new_folder_path, file_name)

#     with open(file_path, 'w') as f:
#         for bbox_li in bbox:
#             x,y,w,h = bbox_li
#             x = x + (w/2)
#             y = y + (h/2)
#             # bbox좌표 정규화
#             x = x / img_w
#             y = y / img_h
#             w = w / img_w
#             h = h / img_h
#             bbox_list = [x,y,w,h]
#             bbox_list = [float(i) for i in bbox_list]
#             bbox_list = [0] + bbox_list
#             result = " ".join(map(str,bbox_list))
#             f.write(" ".join(map(str,bbox_list)) + "\n")

In [ ]:
# # 라벨 폴더생성(초안)
# new_folder_path = Path('./yolo_dataset/resized_labels')
# new_folder_path.mkdir(parents=True, exist_ok=True)
# # 폴더 경로 설정
# txt_folder_path = new_folder_path
# # 라벨 .txt파일 생성
# for file in label_file_list:
#     with open(file, 'r', encoding='utf-8') as f:
#             data = json.load(f)
#             bbox = [i['bbox'] for i in data['annotations']]
#             bbox = [[int(i) for i in li] for li in bbox]
#     file_name = file.split('/')[-1].split('.')[0] + '.txt'
#     file_path = os.path.join(new_folder_path, file_name)

#     with open(file_path, 'w') as f:
#         for bbox_li in bbox:
#             x,y,w,h = bbox_li
#             bbox_list = [0] + [x,y,w,h]
#             result = " ".join(map(str,bbox_list))
#             f.write(" ".join(map(str,bbox_list)) + "\n")

# 4. 이미지 변환체크

In [9]:
# 데이터셋 경로생성함수 
def list_files_in_directory(directory):
    files = []
    for filename in os.listdir(directory):
        filename = os.path.join(directory, filename)
        files.append(filename)
    return files

In [ ]:
# 이미지를 확인할 경로
image_dir = list_files_in_directory(r"C:\AI project\01. Crack Detection\crack_dataset_0504\01_resized_images")
label_dir = list_files_in_directory(r"C:\AI project\01. Crack Detection\crack_dataset_0504\01_resized_labels")

# 인덱스별 이미지 확인함수
def show_640_sample(label_file, image_file, index):
    image = cv2.imread(image_file[index])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # 이미지의 너비와 높이
    image_width, image_height = image.shape[1], image.shape[0]
    
    # bbox좌표
    with open(label_file[index], 'r', encoding='utf-8') as f:
        # 한 줄씩 읽기
        for line in f:
            bbox_info = line.strip().split(' ')
            # 클래스 구분
            class_index = int(bbox_info[0])
            # 바운딩박스 좌표
            x, y, w, h = map(float, bbox_info[1:])
            x *= image_width
            y *= image_height
            w *= image_width
            h *= image_height
            # 정수형 좌표 계산 및 조정
            x1 = int(x - w / 2)
            y1 = int(y - h / 2)
            x2 = int(x + w / 2)
            y2 = int(y + h / 2)
            # 올바른 타입으로 사각형 그리기
            cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 3)
    
    plt.figure(figsize=(6,6))
    plt.imshow(image)
    plt.show()


# 인터랙티브 함수 정의
@interact(index=(0, len(label_dir)-1))
def interact_image(index):
    show_640_sample(label_dir, image_dir, index)

# 5. train/val 나누기

In [ ]:
import os
import random
import shutil

def split_dataset(image_dir, label_dir, train_ratio=0.7):
    # 이미지 및 라벨 파일 리스트 가져오기
    image_files = os.listdir(image_dir)
    label_files = os.listdir(label_dir)
    
    # 이미지 및 라벨 파일 정렬
    image_files.sort()
    label_files.sort()
    
    # 이미지 및 라벨 파일 수
    num_images = len(image_files)
    
    # Train 및 Val 폴더 생성
    os.makedirs('train/images', exist_ok=True)
    os.makedirs('train/labels', exist_ok=True)
    os.makedirs('val/images', exist_ok=True)
    os.makedirs('val/labels', exist_ok=True)
    
    # Train 및 Val 폴더에 이미지 및 라벨 파일 복사
    num_train_images = int(num_images * train_ratio)
    train_image_files = image_files[:num_train_images]
    train_label_files = label_files[:num_train_images]
    val_image_files = image_files[num_train_images:]
    val_label_files = label_files[num_train_images:]
    
    for image_file, label_file in zip(train_image_files, train_label_files):
        shutil.copy(os.path.join(image_dir, image_file), 'train/images')
        shutil.copy(os.path.join(label_dir, label_file), 'train/labels')
        
    for image_file, label_file in zip(val_image_files, val_label_files):
        shutil.copy(os.path.join(image_dir, image_file), 'val/images')
        shutil.copy(os.path.join(label_dir, label_file), 'val/labels')

In [ ]:
# # 원본폴더_image/label(전체)
# dataset_img = r"C:\AI project\01. Crack Detection\yolo_dataset\images"
# dataset_label = r"C:\AI project\01. Crack Detection\yolo_dataset\labels"
# # # 이미지 및 라벨 폴더의 경로 지정(train / val분할)
# # image_dir = './images'
# # label_dir = './labels'
# # 데이터셋 분할 비율 설정하여 데이터셋 분할
# split_dataset(dataset_img, dataset_label, train_ratio=0.9)

In [11]:
# 이미지를 확인할 경로
image_dir = list_files_in_directory("./yolo_dataset/train/images/")
label_dir = list_files_in_directory("./yolo_dataset/train/labels/")

# 인덱스별 이미지 확인함수
def show_640_sample(label_file, image_file, index):
    image = cv2.imread(image_file[index])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # 이미지의 너비와 높이
    image_width, image_height = image.shape[1], image.shape[0]
    
    # bbox좌표
    with open(label_file[index], 'r', encoding='utf-8') as f:
        # 한 줄씩 읽기
        for line in f:
            bbox_info = line.strip().split(' ')
            # 클래스 구분
            class_index = int(bbox_info[0])
            # 바운딩박스 좌표
            x, y, w, h = map(float, bbox_info[1:])
            x *= image_width
            y *= image_height
            w *= image_width
            h *= image_height
            # 정수형 좌표 계산 및 조정
            x1 = int(x - w / 2)
            y1 = int(y - h / 2)
            x2 = int(x + w / 2)
            y2 = int(y + h / 2)
            # 올바른 타입으로 사각형 그리기
            cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 3)
    
    plt.figure(figsize=(6,6))
    plt.imshow(image)
    plt.show()

# 인터랙티브 함수 정의
@interact(index=(0, len(label_dir)-1))
def interact_image(index):
    show_640_sample(label_dir, image_dir, index)

interactive(children=(IntSlider(value=5264, description='index', max=10529), Output()), _dom_classes=('widget-…